In [1]:
import re
from bs4 import BeautifulSoup, Comment

In [2]:
folder = "2017/07/19"
folder

'2017/07/19'

In [3]:
# chrome_match.html
# firefox_player_skill_table.txt
file = open(folder + "/match.html", "r")
soup = BeautifulSoup(file.read(), "html.parser")
file.close()

In [4]:
for div in soup.find_all("div", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_timelineEventPanel"):
    #for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsAway"):
    for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsHome"):
        #print(inputTag.prettify())
        valueString = inputTag.get('value')
        valueString = valueString[1:-1]
        valueString = re.sub('},', '}\n', valueString)
        print("PO,Number,Star,FromMin,ToMin")
        oldPO = ""
        oldToMin = ""
        for line in valueString.split("\n"):
            line = line[1:-1]
            col = line.split(",")
            col[1] = re.sub('"PositionID":', '', col[1])
            col[0] = re.sub('"PlayerId":', '', col[0])
            col[3] = re.sub('"Stars":', '', col[3])
            col[9] = re.sub('"FromMin":', '', col[9])
            col[10] = re.sub('"ToMin":', '', col[10])
            if ( col[1] == "-1" and oldToMin == col[9] ) :
                col[1] = oldPO
            else :
                oldPO = col[1]
                oldToMin = col[10]
            
            # PO rename
            if col[1] == "0" : col[1] = "CO"
            if col[1] == "100" : col[1] = "KP"
            if ( col[1] == "101" or col[1] == "105" ) : col[1] = "WB"
            if ( col[1] == "102" or col[1] == "104" ) : col[1] = "CD"
            if ( col[1] == "106" or col[1] == "110" ) : col[1] = "W"
            if ( col[1] == "107" or col[1] == "109" ) : col[1] = "IM"
            if ( col[1] == "111" or col[1] == "113" ) : col[1] = "FW"
            
            # FromMin ToMin replace
            if col[9] == "-1" : col[9] = "0"
            if col[10] == "-1" : col[10] = "0"
            if col[10] == "92" : col[10] = "90"
                
            print(col[1] + "," + col[0] + "," + col[3] + "," + col[9] + "," + col[10])
                
        #valueString = valueString[1:-1]
        #print(valueString)

PO,Number,Star,FromMin,ToMin
CO,427619232,0.0,0,0
KP,427619233,4.5,0,90
WB,427619238,4.0,0,90
CD,427619235,3.5,0,90
CD,427619239,3.0,0,90
WB,427619240,3.5,0,90
W,427619245,4.5,0,90
IM,427619242,4.5,0,90
IM,427619243,3.5,0,50
IM,425734621,-1.0,50,90
W,428125893,2.5,0,90
FW,427619244,3.5,0,60
FW,427619250,-1.0,60,90
FW,427619248,4.5,0,90
114,428709461,0.0,0,0
115,428112207,0.0,0,0
117,427619247,0.0,0,0


In [ ]:
import psycopg2

In [ ]:
def dropPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """DROP TABLE IF EXISTS player ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
# dropPlayer()

In [ ]:
def createPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
CREATE TABLE player (
    Date DATE, 
    Name VARCHAR(30), 
    Age VARCHAR(10), 
    TSI BIGINT, 
    PayPerWeek BIGINT, 
    Form INT, 
    Stamina INT, 
    Special VARCHAR(8),
    Experience INT,
    Leadership INT,
    Loyalty INT,
    Keeper INT,
    Defending INT,
    Playmaking INT,
    Winger INT,
    Passing INT,
    Scoring INT,
    SetPieces INT
) ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#createPlayer()

In [ ]:
def insertPlayer(folder, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """INSERT INTO player (SELECT to_date(%s, 'YYYYMMDD'), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ;"""
        cur.execute(sql, (folder,row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16]))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def selectPlayer(tableName):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        cur.execute("SELECT * FROM " + tableName)
        row = cur.fetchone()
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#selectPlayer("player")

In [ ]:
import csv

In [ ]:
with open(folder + '/player.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match('^[0-9]{1,2}\. ', row[0])) :
            print(folder, row)
            insertPlayer(folder, row)